In [ ]:
from pytplot import tplot_restore, tplot_names, tplot, get_data
from discontinuitypy import IDsDataset
from space_analysis.ds.meta import Meta, PlasmaMeta, TempMeta
import polars as pl
import numpy as np

In [ ]:
enc = 7
tau = 16
name = 'psp'

In [ ]:
file_name = f"../data/psp_e{enc:02}.tplot"

tplot_restore(file_name)
# sav_data = readsav(file_name, python_dict=True, verbose=True)
# print(sav_data.keys())

In [ ]:
tnames = tplot_names()

In [ ]:
def get_data_lf(name):
    da = get_data(name, xarray=True)
    df = da.to_pandas().reset_index()
    return pl.LazyFrame(df).with_columns(
        pl.col("time").dt.cast_time_unit('us'),
    )

density_col = 'plasma_density'
velocity_cols = ['v_R', 'v_T', 'v_N']

mag_data = get_data_lf('psp_fld_l2_mag_RTN_4_Sa_per_Cyc')
density_data = get_data_lf('psp_swp_spi_sf00_L3_DENS').rename({'0': density_col})
velocity_data = get_data_lf('psp_swp_spi_sf00_L3_VEL_RTN').rename({'0': 'v_R', '1': 'v_T', '2': 'v_N'}) # velocity in RTN in spacecraft frame, regular resolution, you need this to calculate current sheet thickness
ion_temp_data = get_data_lf('Tp_spani_b').rename({'0': 'ion_temp_para', '1': 'ion_temp_perp'})

plasma_data = density_data.join(velocity_data, on='time')

plasma_meta = PlasmaMeta(
    density_col=density_col,
    velocity_cols=velocity_cols,
)

In [ ]:
time = mag_data.collect().get_column("time").to_numpy()
ts = np.median(np.diff(time)).item()

In [ ]:
from utils.temp import calc_pressure_anisotropy

In [ ]:
ids_dataset = IDsDataset(
    name=name,
    mag_data=mag_data,
    tau=tau,
    ts=ts,
    ion_temp_data = ion_temp_data,
    plasma_data=plasma_data,
    plasma_meta=plasma_meta
    
)
ids_dataset.find_events().update_events()

ids_dataset.events = ids_dataset.events.pipe(calc_pressure_anisotropy)

In [ ]:
path = f"../data/enc{enc}/events.PSP.arrow"
ids_dataset.export(path)

In [ ]:

tplot(tnames)